In [125]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main amd64 Packages [36.9 

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

In [0]:
spark.catalog.listTables()

[]

In [127]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD

--2020-02-08 16:51:35--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.199, 52.206.68.26, 52.206.140.205
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

ssType=DOWNLOAD         [             <=>    ] 815.93M  2.76MB/s               ^C


In [130]:
!ls

 reported_crimes.csv		 spark-2.3.1-bin-hadoop2.7.tgz
'rows.csv?accessType=DOWNLOAD'	 spark-2.3.1-bin-hadoop2.7.tgz.1
 sample_data			 spark-warehouse
 spark-2.3.1-bin-hadoop2.7


In [0]:
!mv rows.csv\?accessType\=DOWNLOAD reported_crimes.csv

In [0]:
rc=spark.read.csv('reported_crimes.csv',header=True)

In [0]:
rc.show(2)

+--------+-----------+--------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|                Date|             Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+--------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|01/01/2001 11:00:...|   016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|     

In [0]:
from pyspark.sql.functions import to_timestamp
rc=rc.withColumn('Date',to_timestamp(rc.Date,'MM/dd/yyyy hh:mm:ss'))

In [0]:
rc.show(3)

+--------+-----------+-------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|             Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|   016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|        n

In [0]:
rc.createOrReplaceTempView(name='reported_crimes')

In [0]:
rc=spark.sql("select * from reported_crimes where Date <= '2018-11-11' ")

In [0]:
rc.createOrReplaceTempView(name='reported_crimes')

In [0]:
query_result=spark.sql("select * from reported_crimes where Date>'2018-11-11'")

In [0]:
query_result.show()

+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+--------+---------+--------+
| ID|Case Number|Date|Block|IUCR|Primary Type|Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On|Latitude|Longitude|Location|
+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+--------+---------+--------+
+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+--------+---------+--------+



In [0]:
rc.show(10)

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153|  DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      

In [0]:
limit_result=rc.limit(10)

In [0]:
type(limit_result)

pyspark.sql.dataframe.DataFrame

In [0]:
limit_result.show()

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153|  DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      

In [0]:
head_result=rc.head(10)

In [0]:
head_result

[Row(ID='11034701', Case Number='JA366925', Date=datetime.datetime(2001, 1, 1, 11, 0), Block='016XX E 86TH PL', IUCR='1153', Primary Type='DECEPTIVE PRACTICE', Description='FINANCIAL IDENTITY THEFT OVER $ 300', Location Description='RESIDENCE', Arrest='false', Domestic='false', Beat='0412', District='004', Ward='8', Community Area='45', FBI Code='11', X Coordinate=None, Y Coordinate=None, Year='2001', Updated On='08/05/2017 03:50:08 PM', Latitude=None, Longitude=None, Location=None),
 Row(ID='11227287', Case Number='JB147188', Date=datetime.datetime(2017, 10, 8, 3, 0), Block='092XX S RACINE AVE', IUCR='0281', Primary Type='CRIM SEXUAL ASSAULT', Description='NON-AGGRAVATED', Location Description='RESIDENCE', Arrest='false', Domestic='false', Beat='2222', District='022', Ward='21', Community Area='73', FBI Code='02', X Coordinate=None, Y Coordinate=None, Year='2017', Updated On='02/11/2018 03:57:41 PM', Latitude=None, Longitude=None, Location=None),
 Row(ID='11227583', Case Number='JB147

In [0]:
take_result=rc.take(10)

In [0]:
take_result

[Row(ID='11034701', Case Number='JA366925', Date=datetime.datetime(2001, 1, 1, 11, 0), Block='016XX E 86TH PL', IUCR='1153', Primary Type='DECEPTIVE PRACTICE', Description='FINANCIAL IDENTITY THEFT OVER $ 300', Location Description='RESIDENCE', Arrest='false', Domestic='false', Beat='0412', District='004', Ward='8', Community Area='45', FBI Code='11', X Coordinate=None, Y Coordinate=None, Year='2001', Updated On='08/05/2017 03:50:08 PM', Latitude=None, Longitude=None, Location=None),
 Row(ID='11227287', Case Number='JB147188', Date=datetime.datetime(2017, 10, 8, 3, 0), Block='092XX S RACINE AVE', IUCR='0281', Primary Type='CRIM SEXUAL ASSAULT', Description='NON-AGGRAVATED', Location Description='RESIDENCE', Arrest='false', Domestic='false', Beat='2222', District='022', Ward='21', Community Area='73', FBI Code='02', X Coordinate=None, Y Coordinate=None, Year='2017', Updated On='02/11/2018 03:57:41 PM', Latitude=None, Longitude=None, Location=None),
 Row(ID='11227583', Case Number='JB147

In [0]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [0]:
rc.dtypes

[('ID', 'string'),
 ('Case Number', 'string'),
 ('Date', 'timestamp'),
 ('Block', 'string'),
 ('IUCR', 'string'),
 ('Primary Type', 'string'),
 ('Description', 'string'),
 ('Location Description', 'string'),
 ('Arrest', 'string'),
 ('Domestic', 'string'),
 ('Beat', 'string'),
 ('District', 'string'),
 ('Ward', 'string'),
 ('Community Area', 'string'),
 ('FBI Code', 'string'),
 ('X Coordinate', 'string'),
 ('Y Coordinate', 'string'),
 ('Year', 'string'),
 ('Updated On', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'string'),
 ('Location', 'string')]

In [0]:
from pyspark.sql.types import IntegerType,StringType,StructField,StructType


In [0]:
rc.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [0]:
someSchema=StructType([
                       StructField('some_column_name',StringType(),False)#columnName,DataType(),nullable=false
])

In [0]:
someSchema

StructType(List(StructField(some_column_name,StringType,false)))

In [0]:
rc.show()

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153|  DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      

In [0]:
sample=rc.limit(30)

In [0]:
sample.show(30)

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153|  DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      

In [0]:
sample.groupBy('Primary Type','Description').count().show()#counts of all occurred combinations of Primary Type and Descripton

+--------------------+--------------------+-----+
|        Primary Type|         Description|count|
+--------------------+--------------------+-----+
|  DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|    6|
| CRIM SEXUAL ASSAULT|      NON-AGGRAVATED|    2|
|            BURGLARY|      UNLAWFUL ENTRY|    1|
|               THEFT|           OVER $500|    3|
| CRIM SEXUAL ASSAULT|           PREDATORY|    4|
|OFFENSE INVOLVING...|CRIM SEX ABUSE BY...|    1|
|OFFENSE INVOLVING...|AGG SEX ASSLT OF ...|    2|
|     CRIMINAL DAMAGE|         TO PROPERTY|    1|
|       OTHER OFFENSE|HARASSMENT BY ELE...|    1|
|               THEFT|      $500 AND UNDER|    1|
|OFFENSE INVOLVING...|AGG CRIM SEX ABUS...|    1|
|  DECEPTIVE PRACTICE|   CREDIT CARD FRAUD|    1|
|OFFENSE INVOLVING...|SEX ASSLT OF CHIL...|    2|
|  DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|    3|
|               THEFT|       FROM BUILDING|    1|
+--------------------+--------------------+-----+



In [0]:
import pyspark.sql.functions as F

In [0]:
sample.groupBy('Primary Type').agg(F.count('Primary Type').alias('count')).show()

+--------------------+-----+
|        Primary Type|count|
+--------------------+-----+
|  DECEPTIVE PRACTICE|   10|
| CRIM SEXUAL ASSAULT|    6|
|            BURGLARY|    1|
|               THEFT|    5|
|OFFENSE INVOLVING...|    6|
|     CRIMINAL DAMAGE|    1|
|       OTHER OFFENSE|    1|
+--------------------+-----+



In [0]:
sample.groupBy('Primary Type').count().show()

+--------------------+-----+
|        Primary Type|count|
+--------------------+-----+
|  DECEPTIVE PRACTICE|   10|
| CRIM SEXUAL ASSAULT|    6|
|            BURGLARY|    1|
|               THEFT|    5|
|OFFENSE INVOLVING...|    6|
|     CRIMINAL DAMAGE|    1|
|       OTHER OFFENSE|    1|
+--------------------+-----+



In [0]:
sample.createOrReplaceTempView(name='sample')

In [0]:
spark.catalog.listTables()

[Table(name='reported_crimes', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='sample', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [0]:
sample.groupBy('Primary Type').count().show()

+--------------------+-----+
|        Primary Type|count|
+--------------------+-----+
|  DECEPTIVE PRACTICE|   10|
| CRIM SEXUAL ASSAULT|    6|
|            BURGLARY|    1|
|               THEFT|    5|
|OFFENSE INVOLVING...|    6|
|     CRIMINAL DAMAGE|    1|
|       OTHER OFFENSE|    1|
+--------------------+-----+



In [0]:
import pandas as pd
someData={
    'name':['Hamza','Neemah','Hamza','Hamza','Neemah'],
    'score':[23,1,34,32,1]
}

In [0]:
df=pd.DataFrame(someData)

In [0]:
df.head()

,name,score
0,Hamza,23
1,Neemah,1
2,Hamza,34
3,Hamza,32
4,Neemah,1


In [0]:
sparksample=spark.createDataFrame(data=df)

In [0]:
sparksample.show()

+------+-----+
|  name|score|
+------+-----+
| Hamza|   23|
|Neemah|    1|
| Hamza|   34|
| Hamza|   32|
|Neemah|    1|
+------+-----+



In [0]:
sparksample.groupBy('name').avg().show()

+------+------------------+
|  name|        avg(score)|
+------+------------------+
|Neemah|               1.0|
| Hamza|29.666666666666668|
+------+------------------+



In [0]:
sample.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [0]:
sample.select(sample.columns[:3]).show()

+--------+-----------+-------------------+
|      ID|Case Number|               Date|
+--------+-----------+-------------------+
|11034701|   JA366925|2001-01-01 11:00:00|
|11227287|   JB147188|2017-10-08 03:00:00|
|11227583|   JB147595|2017-03-28 02:00:00|
|11227293|   JB147230|2017-09-09 08:17:00|
|11227634|   JB147599|2017-08-26 10:00:00|
|11227517|   JB138481|2013-02-10 00:00:00|
|11227503|   JB146383|2015-01-01 00:01:00|
|11227508|   JB146365|2017-01-01 00:01:00|
|11022695|   JA353568|2017-07-17 10:10:00|
|11227633|   JB147500|2017-12-28 03:55:00|
|11227586|   JB147613|2017-02-10 00:00:00|
|11227475|   JB147314|2017-11-22 02:42:00|
|11227247|   JB147078|2012-01-01 09:00:00|
|11036284|   JA370282|2017-07-29 03:40:00|
|11227509|   JB146413|2017-01-22 00:01:00|
|11227407|   JB147329|2017-10-14 00:01:00|
|11028056|   JA359834|2014-10-15 03:00:00|
|11028299|   JA360073|2015-05-30 00:00:00|
|11028290|   JA360134|2015-05-30 00:00:00|
|11041422|   JA375398|2017-07-29 10:00:00|
+--------+-

In [0]:
sample.select(['ID']).show(2)

+--------+
|      ID|
+--------+
|11034701|
|11227287|
+--------+
only showing top 2 rows



In [0]:
print('hello')

hello


In [0]:
spark.catalog.listTables()

[Table(name='reported_crimes', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='sample', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [0]:
sample=sample.withColumnRenamed('ID','id')

In [0]:
sample.show(1)

+--------+-----------+-------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      id|Case Number|               Date|          Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|016XX E 86TH PL|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|        null|        null

In [0]:
sample=sample.drop('X Coordinate','Y Coordinate','Latitude','Longitude')

In [0]:
sample.show(1)

+--------+-----------+-------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+--------+
|      id|Case Number|               Date|          Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|Location|
+--------+-----------+-------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|016XX E 86TH PL|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|2001|08/05/2017 03:50:...|    null|
+--------+-----------+-------------------+---------------+----+------------------+--------------------+--------------------+

In [0]:
sample=sample.drop('Location')
sample.show()

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|      id|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153|  DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|2001|08/05/2017 03:50:...|
|11227287|   JB147188|2017-10-08 03:00:00|  092XX S RACINE AVE|0281| CRIM SEXUAL ASSAULT|      NON-AGGRAVATED|           RESIDENCE| 

In [0]:
sample=sample.fillna({'Block':'Somewhere'})
sample.show()

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|      id|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153|  DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|2001|08/05/2017 03:50:...|
|11227287|   JB147188|2017-10-08 03:00:00|  092XX S RACINE AVE|0281| CRIM SEXUAL ASSAULT|      NON-AGGRAVATED|           RESIDENCE| 

In [0]:
sample=sample.na.drop(subset=[col for col in sample.columns])

In [0]:
sample.show(1)

+--------+-----------+-------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|      id|Case Number|               Date|          Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|
+--------+-----------+-------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|11034701|   JA366925|2001-01-01 11:00:00|016XX E 86TH PL|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|2001|08/05/2017 03:50:...|
+--------+-----------+-------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+-

In [0]:
sample.select('Case Number','Date','Arrest').show(4)

+-----------+-------------------+------+
|Case Number|               Date|Arrest|
+-----------+-------------------+------+
|   JA366925|2001-01-01 11:00:00| false|
|   JB147188|2017-10-08 03:00:00| false|
|   JB147595|2017-03-28 02:00:00| false|
|   JB147230|2017-09-09 08:17:00| false|
+-----------+-------------------+------+
only showing top 4 rows



In [0]:
from pyspark.sql.functions import col

In [0]:
rc.select('*').where("Date = '2017-08-17'").groupBy('Latitude').count().orderBy(col('count'),ascending=False).show()

+------------+-----+
|    Latitude|count|
+------------+-----+
|        null|    2|
| 41.74644628|    1|
|41.691352112|    1|
|41.895676734|    1|
|41.854917247|    1|
|41.842653676|    1|
|41.890045974|    1|
|41.977269698|    1|
|41.765078582|    1|
|41.877460016|    1|
|41.981247027|    1|
|41.934778833|    1|
|41.876140547|    1|
|41.773275343|    1|
|41.998697566|    1|
|41.745102603|    1|
|41.798838278|    1|
|41.747236315|    1|
|42.014911903|    1|
|41.878864086|    1|
+------------+-----+
only showing top 20 rows



In [0]:
someData

{'name': ['Hamza', 'Neemah', 'Hamza', 'Hamza', 'Neemah'],
 'score': [23, 1, 34, 32, 1]}

In [0]:
sample.describe().show()

+-------+-----------------+-----------+--------------------+------------------+------------+--------------+--------------------+------+--------+------------------+------------------+------------------+------------------+-----------------+------------------+--------------------+
|summary|               id|Case Number|               Block|              IUCR|Primary Type|   Description|Location Description|Arrest|Domestic|              Beat|          District|              Ward|    Community Area|         FBI Code|              Year|          Updated On|
+-------+-----------------+-----------+--------------------+------------------+------------+--------------+--------------------+------+--------+------------------+------------------+------------------+------------------+-----------------+------------------+--------------------+
|  count|               28|         28|                  28|                28|          28|            28|                  28|    28|      28|                28|

In [0]:
rc.groupBy('Location Description','Primary Type','Description').count().orderBy(col('count'),ascending=True).show()

+--------------------+--------------------+--------------------+-----+
|Location Description|        Primary Type|         Description|count|
+--------------------+--------------------+--------------------+-----+
|      DELIVERY TRUCK|  DECEPTIVE PRACTICE|STOLEN PROP: BUY/...|    1|
|VEHICLE-COMMERCIA...|OFFENSE INVOLVING...|         CHILD ABUSE|    1|
|           RESIDENCE|       OTHER OFFENSE|ARSONIST: FAIL TO...|    1|
|   CONVENIENCE STORE|  DECEPTIVE PRACTICE|UNLAWFUL USE OF A...|    1|
|           CTA TRAIN|  DECEPTIVE PRACTICE|   COUNTERFEIT CHECK|    1|
|AUTO / BOAT / RV ...|  DECEPTIVE PRACTICE|DECEPTIVE COLLECT...|    1|
|          RESTAURANT|        PROSTITUTION|OTHER PROSTITUTIO...|    1|
|LAKEFRONT/WATERFR...|         SEX OFFENSE|SEXUAL EXPLOITATI...|    1|
|RESIDENCE PORCH/H...| MOTOR VEHICLE THEFT|THEFT/RECOVERY: T...|    1|
|COMMERCIAL / BUSI...|         SEX OFFENSE|              BIGAMY|    1|
|  HIGHWAY/EXPRESSWAY|             ROBBERY|AGGRAVATED VEHICU...|    1|
|     

In [0]:
someData

{'name': ['Hamza', 'Neemah', 'Hamza', 'Hamza', 'Neemah'],
 'score': [23, 1, 34, 32, 1]}

In [0]:
rc.groupBy('Primary Type').count().show()

+--------------------+-------+
|        Primary Type|  count|
+--------------------+-------+
|OFFENSE INVOLVING...|  45682|
|            STALKING|   3384|
|PUBLIC PEACE VIOL...|  47785|
|           OBSCENITY|    582|
|NON-CRIMINAL (SUB...|      9|
|               ARSON|  11156|
|   DOMESTIC VIOLENCE|      1|
|            GAMBLING|  14422|
|   CRIMINAL TRESPASS| 193370|
|             ASSAULT| 418502|
|      NON - CRIMINAL|     38|
|LIQUOR LAW VIOLATION|  14067|
| MOTOR VEHICLE THEFT| 314143|
|               THEFT|1418415|
|             BATTERY|1232212|
|             ROBBERY| 255601|
|            HOMICIDE|   9464|
|           RITUALISM|     23|
|    PUBLIC INDECENCY|    161|
| CRIM SEXUAL ASSAULT|  27380|
+--------------------+-------+
only showing top 20 rows



In [0]:
rc.show(2)

+--------+-----------+-------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|             Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|   016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|        n

In [0]:
arrests=rc.groupby('Arrest').count().show()

+------+-------+
|Arrest|  count|
+------+-------+
| false|4878274|
|  true|1873996|
+------+-------+



In [0]:
rc.groupBy('Location Description').count().orderBy(col('count'),ascending=False).show(10)

+--------------------+-------+
|Location Description|  count|
+--------------------+-------+
|              STREET|1770566|
|           RESIDENCE|1144482|
|           APARTMENT| 698082|
|            SIDEWALK| 665519|
|               OTHER| 256898|
|PARKING LOT/GARAG...| 193829|
|               ALLEY| 150905|
|SCHOOL, PUBLIC, B...| 142338|
|    RESIDENCE-GARAGE| 131648|
|  SMALL RETAIL STORE| 119269|
+--------------------+-------+
only showing top 10 rows



In [0]:
rc.select('Arrest').count()

6752270

In [0]:
rc.count()

6752270

In [128]:
print('hello')

hello


In [0]:
import pyspark.sql.functions as F

In [131]:
rc.show(1)

+--------+-----------+-------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|          Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|016XX E 86TH PL|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|        null|        null

In [132]:
print(dir(F))

['AutoBatchedSerializer', 'Column', 'DataFrame', 'DataType', 'PandasUDFType', 'PickleSerializer', 'PythonEvalType', 'SparkContext', 'StringType', 'UserDefinedFunction', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_binary_mathfunctions', '_collect_list_doc', '_collect_set_doc', '_create_binary_mathfunction', '_create_function', '_create_udf', '_create_window_function', '_functions', '_functions_1_4', '_functions_1_6', '_functions_2_1', '_functions_deprecated', '_lit_doc', '_message', '_string_functions', '_test', '_to_java_column', '_to_seq', '_window_functions', '_wrap_deprecated_function', 'abs', 'acos', 'add_months', 'approxCountDistinct', 'approx_count_distinct', 'array', 'array_contains', 'asc', 'ascii', 'asin', 'atan', 'atan2', 'avg', 'base64', 'bin', 'bitwiseNOT', 'blacklist', 'broadcast', 'bround', 'cbrt', 'ceil', 'coalesce', 'col', 'collect_list', 'collect_set', 'column', 'concat', 'concat_ws', 'conv', 'c

Display the Primary Type Column in lower and upper characters an the first 4 characters of the colum.

In [133]:
sample.count()

28

In [0]:
sample=sample.withColumn('Primary Type',F.lower(col('Primary Type')))

In [149]:
sample.show(2)

+--------+-----------+-------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+-------------------+
|      id|Case Number|               Date|             Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On| lower_primary_type|
+--------+-----------+-------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+-------------------+
|11034701|   JA366925|2001-01-01 11:00:00|   016XX E 86TH PL|1153| deceptive practice|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|2001|08/05/2017 03:50:...| deceptive practice|
|11227287|   JB147188|2017-10-08 03:00:00|092XX S RACINE AVE|028

In [0]:
sample=sample.drop('lower_primary_type')

In [0]:
from pyspark.sql.types import StringType
def somestringfunction(x):
  return x.lower()[:2]

udfFunction=F.udf(somestringfunction,StringType())


In [0]:
sample=sample.withColumn('testudf',udfFunction("Description"))

In [153]:
sample.show()

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+-------+
|      id|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|testudf|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+-------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153|  deceptive practice|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|2001|08/05/2017 03:50:...|     fi|
|11227287|   JB147188|2017-10-08 03:00:00|  092XX S RACINE AVE|0281| crim sexual assault|      NON-A

In [0]:
sample=sample.drop('testudf')

In [157]:
sample.show(2)

+--------+-----------+-------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|      id|Case Number|               Date|             Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|
+--------+-----------+-------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|11034701|   JA366925|2001-01-01 11:00:00|   016XX E 86TH PL|1153| deceptive practice|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|2001|08/05/2017 03:50:...|
|11227287|   JB147188|2017-10-08 03:00:00|092XX S RACINE AVE|0281|crim sexual assault|      NON-AGGRAVATED|           RESIDENCE| false|   false|

In [158]:
sample.printSchema()

root
 |-- id: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = false)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)



In [159]:
help(F.window)

Help on function window in module pyspark.sql.functions:

window(timeColumn, windowDuration, slideDuration=None, startTime=None)
    Bucketize rows into one or more time windows given a timestamp specifying column. Window
    starts are inclusive but the window ends are exclusive, e.g. 12:05 will be in the window
    [12:05,12:10) but not in [12:00,12:05). Windows can support microsecond precision. Windows in
    the order of months are not supported.
    
    The time column must be of :class:`pyspark.sql.types.TimestampType`.
    
    Durations are provided as strings, e.g. '1 second', '1 day 12 hours', '2 minutes'. Valid
    interval strings are 'week', 'day', 'hour', 'minute', 'second', 'millisecond', 'microsecond'.
    If the ``slideDuration`` is not provided, the windows will be tumbling windows.
    
    The startTime is the offset with respect to 1970-01-01 00:00:00 UTC with which to start
    window intervals. For example, in order to have hourly tumbling windows that start 15

In [0]:
sample=sample.orderBy('Date')

In [161]:
sample.show(10)

+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|      id|Case Number|               Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|          Updated On|
+--------+-----------+-------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+--------------------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153|  deceptive practice|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|2001|08/05/2017 03:50:...|
|11042183|   JA376593|2001-07-12 00:01:00|    024XX N LUNA AVE|0266| crim sexual assault|           PREDATORY|           RESIDENCE| 

In [0]:
another_way=sample.select(udfFunction('Case Number'))

In [164]:
another_way.show()

+-------------------------------+
|somestringfunction(Case Number)|
+-------------------------------+
|                             ja|
|                             ja|
|                             ja|
|                             ja|
|                             ja|
|                             jb|
|                             ja|
|                             jb|
|                             ja|
|                             jb|
|                             ja|
|                             jb|
|                             ja|
|                             ja|
|                             ja|
|                             jb|
|                             jb|
|                             jb|
|                             jb|
|                             ja|
+-------------------------------+
only showing top 20 rows



In [0]:
something=rc.select(F.min(col('Date')))

In [168]:
something.show()

+-------------------+
|          min(Date)|
+-------------------+
|2001-01-01 00:00:00|
+-------------------+



In [0]:
something=rc.select(F.max('Date'))

In [172]:
something.show()

+-------------------+
|          max(Date)|
+-------------------+
|2018-11-10 11:58:00|
+-------------------+



In [174]:
something=sample.select(F.date_sub(F.min(col('Date')),3)).show()

+----------------------+
|date_sub(min(Date), 3)|
+----------------------+
|            2000-12-29|
+----------------------+



In [0]:
df=spark.createDataFrame([('2019-12-25 13:30:00',)],['Christmas'])
#createDataframe([(row1 of col1,row2 of col1)],[col1 name])

In [177]:
df.show()

+-------------------+
|          Christmas|
+-------------------+
|2019-12-25 13:30:00|
+-------------------+



In [0]:
from pyspark.sql.functions import to_timestamp,to_date

In [183]:
df.select(to_date('Christmas',format='yyyy-MM-dd HH:mm:ss')).show()

+-------------------------------------------+
|to_date(`Christmas`, 'yyyy-MM-dd HH:mm:ss')|
+-------------------------------------------+
|                                 2019-12-25|
+-------------------------------------------+



In [184]:
df.select(to_timestamp('Christmas',format='yyyy-MM-dd HH:mm:ss')).show()

+------------------------------------------------+
|to_timestamp(`Christmas`, 'yyyy-MM-dd HH:mm:ss')|
+------------------------------------------------+
|                             2019-12-25 13:30:00|
+------------------------------------------------+



In [185]:
rc.show(1)

+--------+-----------+-------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|          Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|016XX E 86TH PL|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|        null|        null

In [0]:
rc=rc.withColumn('Date',F.to_date('Date','yyyy-MM-dd HH:mm:ss'))

In [187]:
rc.show()

+--------+-----------+----------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|      Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+----------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01|     016XX E 86TH PL|1153|  DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|        null|        null|2001|08

In [188]:
rc.cache()

DataFrame[ID: string, Case Number: string, Date: date, Block: string, IUCR: string, Primary Type: string, Description: string, Location Description: string, Arrest: string, Domestic: string, Beat: string, District: string, Ward: string, Community Area: string, FBI Code: string, X Coordinate: string, Y Coordinate: string, Year: string, Updated On: string, Latitude: string, Longitude: string, Location: string]

In [189]:
rc.count()

6752270

In [191]:
rc.show(2)

+--------+-----------+----------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|      Date|             Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+----------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01|   016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|        null|        null|2001|08/05/2017 03:

In [192]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [193]:
rc.where("Date < '2015-01-01'").show()

+--------+-----------+----------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|      Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+----------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01|     016XX E 86TH PL|1153|  DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|        null|        null|2001|08